In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy import stats
import os
os.environ['OMP_NUM_THREADS'] = '1'
import matplotlib
matplotlib.style.use('ggplot')

In [4]:
pd.__version__

'2.1.4'

Methodology plan:

Classification model to predict 

In [5]:
train_df= pd.read_csv('train.csv')
test_df= pd.read_csv('test.csv')
misconcept_map= pd.read_csv('misconception_mapping.csv')
sample_submission=pd.read_csv('sample_submission.csv')

In [6]:
print(train_df.shape)
print(test_df.shape)

(1869, 15)
(3, 11)


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1869 entries, 0 to 1868
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   QuestionId        1869 non-null   int64  
 1   ConstructId       1869 non-null   int64  
 2   ConstructName     1869 non-null   object 
 3   SubjectId         1869 non-null   int64  
 4   SubjectName       1869 non-null   object 
 5   CorrectAnswer     1869 non-null   object 
 6   QuestionText      1869 non-null   object 
 7   AnswerAText       1869 non-null   object 
 8   AnswerBText       1869 non-null   object 
 9   AnswerCText       1869 non-null   object 
 10  AnswerDText       1869 non-null   object 
 11  MisconceptionAId  1135 non-null   float64
 12  MisconceptionBId  1118 non-null   float64
 13  MisconceptionCId  1080 non-null   float64
 14  MisconceptionDId  1037 non-null   float64
dtypes: float64(4), int64(3), object(8)
memory usage: 219.2+ KB


In [8]:
train_df.CorrectAnswer.value_counts()

CorrectAnswer
C    488
A    482
B    461
D    438
Name: count, dtype: int64

In [9]:
train_df.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   QuestionId     3 non-null      int64 
 1   ConstructId    3 non-null      int64 
 2   ConstructName  3 non-null      object
 3   SubjectId      3 non-null      int64 
 4   SubjectName    3 non-null      object
 5   CorrectAnswer  3 non-null      object
 6   QuestionText   3 non-null      object
 7   AnswerAText    3 non-null      object
 8   AnswerBText    3 non-null      object
 9   AnswerCText    3 non-null      object
 10  AnswerDText    3 non-null      object
dtypes: int64(3), object(8)
memory usage: 396.0+ bytes


In [11]:
misconcept_map.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


In [12]:
misconDict={}
for i in range(misconcept_map.shape[0]):
    misconDict[misconcept_map.MisconceptionId.values[i]] = misconcept_map.MisconceptionName.values[i]

In [13]:
misconDict

{0: 'Does not know that angles in a triangle sum to 180 degrees',
 1: 'Uses dividing fractions method for multiplying fractions',
 2: 'Believes there are 100 degrees in a full turn',
 3: 'Thinks a quadratic without a non variable term, can not be factorised',
 4: 'Believes addition of terms and powers of terms are equivalent e.g. a + c = a^c',
 5: 'When measuring a reflex angle, gives the acute or obtuse angle that sums to 360 instead',
 6: 'Can identify the multiplier used to form an equivalent fraction but does not apply to the numerator',
 7: 'Believes gradient = change in y',
 8: 'Student thinks that any two angles along a straight line are equal',
 9: 'Thinks there are 180 degrees in a full turn',
 10: 'Believes duration can be read from a timetable, rather than calculating from start and end times',
 11: 'When reading value from graph, reads from the wrong axes.',
 12: 'Thinks that the side view does not include the furthest rows back',
 13: 'Does not subtract from the hours, whe

In [14]:
def cumMergeDictionary(dict_1, dict_2):
   dict_3 = {**dict_1, **dict_2}
   for key, value in dict_3.items():
       if key in dict_1 and key in dict_2:
               dict_3[key] = value + dict_1[key]
   return dict_3

In [15]:
misconception_cols=['MisconceptionAId','MisconceptionBId','MisconceptionCId','MisconceptionDId']
total_of_miscon= {}

for col in misconception_cols:
    temp_dict= train_df[col].value_counts().to_dict()
    total_of_miscon=cumMergeDictionary(temp_dict,total_of_miscon)

In [16]:
total_of_miscon

{1214.0: 54,
 1379.0: 43,
 113.0: 15,
 1880.0: 32,
 2392.0: 27,
 1597.0: 27,
 2481.0: 19,
 2330.0: 11,
 1990.0: 33,
 1248.0: 22,
 690.0: 13,
 220.0: 22,
 1450.0: 5,
 31.0: 16,
 2352.0: 7,
 1383.0: 13,
 2532.0: 7,
 2359.0: 19,
 1130.0: 5,
 1507.0: 36,
 907.0: 10,
 255.0: 10,
 1828.0: 12,
 1055.0: 12,
 618.0: 4,
 2316.0: 38,
 172.0: 14,
 256.0: 6,
 2113.0: 11,
 161.0: 14,
 1416.0: 12,
 120.0: 9,
 1510.0: 14,
 1180.0: 13,
 329.0: 12,
 1103.0: 4,
 108.0: 16,
 699.0: 12,
 219.0: 5,
 2472.0: 3,
 77.0: 22,
 2454.0: 5,
 2362.0: 6,
 61.0: 8,
 547.0: 10,
 2262.0: 6,
 2143.0: 14,
 340.0: 11,
 346.0: 4,
 82.0: 8,
 974.0: 18,
 11.0: 12,
 1338.0: 8,
 2355.0: 19,
 1773.0: 9,
 1527.0: 14,
 725.0: 4,
 512.0: 8,
 167.0: 3,
 2306.0: 7,
 1318.0: 5,
 563.0: 5,
 1198.0: 16,
 217.0: 16,
 134.0: 6,
 2234.0: 7,
 1988.0: 17,
 583.0: 8,
 550.0: 7,
 2252.0: 4,
 1660.0: 4,
 1601.0: 3,
 1542.0: 6,
 2264.0: 4,
 990.0: 4,
 1686.0: 4,
 1072.0: 20,
 105.0: 10,
 110.0: 8,
 357.0: 5,
 657.0: 4,
 1863.0: 2,
 1839.0: 6,
 1

In [17]:
def mergeDictionary(dict_1, dict_2):
   dict_3 = {**dict_1, **dict_2}
   for key, value in dict_3.items():
       if key in dict_1 and key in dict_2:
               dict_3[key] = [dict_1[key],value ]
   return dict_3

In [18]:
MisconDictwTotal= mergeDictionary(total_of_miscon,misconDict)
MisconDictwTotal

{1214.0: [54,
  'When solving an equation, uses the same operation rather than the inverse.'],
 1379.0: [43, 'Rounds down instead of up'],
 113.0: [15, 'Does not recognise difference of two squares'],
 1880.0: [32, 'Mixes up greater than and less than symbols'],
 2392.0: [27, 'Rounds to the wrong degree of accuracy (rounds too much)'],
 1597.0: [27, 'Believes multiplying two negatives gives a negative answer'],
 2481.0: [19,
  'When multiplying decimals, divides by the wrong power of 10 when reinserting the decimal'],
 2330.0: [11, 'Rounds incorrectly by changing multiple place values'],
 1990.0: [33, 'Fails to reflect across mirror line'],
 1248.0: [22, 'Rounds to the wrong degree of accuracy (rounds too little)'],
 690.0: [13, 'Struggles to visualise nets of 3D shapes'],
 220.0: [22, 'Only multiplies the first term in the expansion of a bracket'],
 1450.0: [5, 'Does not understand how to apply sohcahtoa'],
 31.0: [16, 'Does not understand bar modelling in algebra'],
 2352.0: [7, 'Wri

# Preprocessing

In [19]:
train_df['QuestionID_AnswerA']=train_df['QuestionId'].astype(str)+'_A'
train_df['QuestionID_AnswerB']=train_df['QuestionId'].astype(str)+'_B'
train_df['QuestionID_AnswerC']=train_df['QuestionId'].astype(str)+'_C'
train_df['QuestionID_AnswerD']=train_df['QuestionId'].astype(str)+'_D'
train_df.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId,QuestionID_AnswerA,QuestionID_AnswerB,QuestionID_AnswerC,QuestionID_AnswerD
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0,0_A,0_B,0_C,0_D
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN,1_A,1_B,1_C,1_D
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0,2_A,2_B,2_C,2_D
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0,3_A,3_B,3_C,3_D
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0,4_A,4_B,4_C,4_D


In [20]:
test_df['QuestionID_AnswerA']=test_df['QuestionId'].astype(str)+'_A'
test_df['QuestionID_AnswerB']=test_df['QuestionId'].astype(str)+'_B'
test_df['QuestionID_AnswerC']=test_df['QuestionId'].astype(str)+'_C'
test_df['QuestionID_AnswerD']=test_df['QuestionId'].astype(str)+'_D'
test_df.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,QuestionID_AnswerA,QuestionID_AnswerB,QuestionID_AnswerC,QuestionID_AnswerD
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,1869_A,1869_B,1869_C,1869_D
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,1870_A,1870_B,1870_C,1870_D
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1871_A,1871_B,1871_C,1871_D


In [21]:
#this function is to take the test data and make observations out of each incorrect answer instead of each question
def expand_question_data(df,common_cols,mapping):
    expanded_df= pd.DataFrame(columns=common_cols +['MisconceptionID', 'AnswerText', 'QuestionID_Answer'])
    for i in range(0,df.shape[0]):
        if df.iloc[[i]]['CorrectAnswer'].values!= 'A':
            #print(df.iloc[[i]][common_cols +['MisconceptionAId','AnswerAText']].rename(mapping))
            expanded_df= pd.concat([expanded_df,train_df.iloc[[i]][common_cols +['MisconceptionAId','AnswerAText','QuestionID_AnswerA' ]].rename(mapper=mapping,axis=1)])
        if df.iloc[[i]]['CorrectAnswer'].values!= 'B':
            #print(df.iloc[[i]][common_cols +['MisconceptionBId','AnswerBText']].rename(mapping))
            expanded_df= pd.concat([expanded_df,train_df.iloc[[i]][common_cols +['MisconceptionBId','AnswerBText','QuestionID_AnswerB']].rename(mapper=mapping,axis=1)])
        if train_df.iloc[[i]]['CorrectAnswer'].values!= 'C':
            #print(df.iloc[[i]][common_cols +['MisconceptionCId','AnswerCText']].rename(mapping))
            expanded_df= pd.concat([expanded_df,train_df.iloc[[i]][common_cols +['MisconceptionCId','AnswerCText','QuestionID_AnswerC']].rename(mapper=mapping,axis=1)])
        if df.iloc[[i]]['CorrectAnswer'].values!= 'D':
            #print(df.iloc[[i]][common_cols +['MisconceptionDId','AnswerDText']].rename(mapping))
            expanded_df= pd.concat([expanded_df,train_df.iloc[[i]][common_cols +['MisconceptionDId','AnswerDText','QuestionID_AnswerD']].rename(mapper=mapping,axis=1)])

    return expanded_df

In [22]:
common_cols= ["QuestionId",
              "QuestionText",
              "ConstructId",
              "ConstructName",
              "SubjectId",
              "SubjectName",
              "CorrectAnswer"
             ]

In [23]:
mapping= {'MisconceptionAId': 'MisconceptionID',
          'MisconceptionBId': 'MisconceptionID',
          'MisconceptionCId': 'MisconceptionID',
          'MisconceptionDId': 'MisconceptionID',
          'AnswerAText': 'AnswerText',
          'AnswerBText': 'AnswerText',
          'AnswerCText': 'AnswerText',
          'AnswerDText': 'AnswerText',
          'QuestionID_AnswerA': 'QuestionID_Answer',
          'QuestionID_AnswerB': 'QuestionID_Answer',
          'QuestionID_AnswerC': 'QuestionID_Answer',
          'QuestionID_AnswerD': 'QuestionID_Answer'
         }

In [24]:
expanded_df=expand_question_data(train_df,common_cols,mapping)

C:\Users\willv\AppData\Local\Temp\ipykernel_8716\3472321129.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  expanded_df= pd.concat([expanded_df,train_df.iloc[[i]][common_cols +['MisconceptionDId','AnswerDText','QuestionID_AnswerD']].rename(mapper=mapping,axis=1)])


In [25]:
expanded_df

,QuestionId,QuestionText,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,MisconceptionID,AnswerText,QuestionID_Answer
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times 2+(4-5) \),0_B
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times(2+4-5) \),0_C
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,1672.0,Does not need brackets,0_D
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,2142.0,\( m+1 \),1_A
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,143.0,\( m+2 \),1_B
...,...,...,...,...,...,...,...,...,...,...
1867,1867,Tom and Katie are discussing congruence and si...,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,2312.0,Both Tom and Katie,1867_C
1867,1867,Tom and Katie are discussing congruence and si...,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,2312.0,Neither is correct,1867_D
1868,1868,Jo and Paul are arguing about how to fully des...,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,B,801.0,Only\nJo,1868_A
1868,1868,Jo and Paul are arguing about how to fully des...,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,B,801.0,Both Jo and Paul,1868_C


In [26]:
expanded_df.isnull().sum()

QuestionId              0
QuestionText            0
ConstructId             0
ConstructName           0
SubjectId               0
SubjectName             0
CorrectAnswer           0
MisconceptionID      1237
AnswerText              0
QuestionID_Answer       0
dtype: int64

In [27]:
expanded_df['AllText']= expanded_df.apply(lambda x: f"<Construct>{x['ConstructName']} <Subject>{x['SubjectName']} <Question>{x['QuestionText']} <Answer>{x['AnswerText']}",axis=1)
expanded_df

,QuestionId,QuestionText,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,MisconceptionID,AnswerText,QuestionID_Answer,AllText
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times 2+(4-5) \),0_B,<Construct>Use the order of operations to carr...
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times(2+4-5) \),0_C,<Construct>Use the order of operations to carr...
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,1672.0,Does not need brackets,0_D,<Construct>Use the order of operations to carr...
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,2142.0,\( m+1 \),1_A,<Construct>Simplify an algebraic fraction by f...
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,143.0,\( m+2 \),1_B,<Construct>Simplify an algebraic fraction by f...
...,...,...,...,...,...,...,...,...,...,...,...
1867,1867,Tom and Katie are discussing congruence and si...,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,2312.0,Both Tom and Katie,1867_C,<Construct>Distinguish between congruency and ...
1867,1867,Tom and Katie are discussing congruence and si...,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,2312.0,Neither is correct,1867_D,<Construct>Distinguish between congruency and ...
1868,1868,Jo and Paul are arguing about how to fully des...,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,B,801.0,Only\nJo,1868_A,<Construct>Describe a 90° or 270° rotation giv...
1868,1868,Jo and Paul are arguing about how to fully des...,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,B,801.0,Both Jo and Paul,1868_C,<Construct>Describe a 90° or 270° rotation giv...


In [28]:
expanded_test=expand_question_data(test_df,common_cols,mapping)

C:\Users\willv\AppData\Local\Temp\ipykernel_8716\3472321129.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  expanded_df= pd.concat([expanded_df,train_df.iloc[[i]][common_cols +['MisconceptionDId','AnswerDText','QuestionID_AnswerD']].rename(mapper=mapping,axis=1)])


In [29]:
expanded_test['AllText']= expanded_test.apply(lambda x: f"<Construct>{x['ConstructName']} <Subject>{x['SubjectName']} <Question>{x['QuestionText']} <Answer>{x['AnswerText']}",axis=1)
expanded_test

,QuestionId,QuestionText,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,MisconceptionID,AnswerText,QuestionID_Answer,AllText
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times 2+(4-5) \),0_B,<Construct>Use the order of operations to carr...
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times(2+4-5) \),0_C,<Construct>Use the order of operations to carr...
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,1672.0,Does not need brackets,0_D,<Construct>Use the order of operations to carr...
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,2142.0,\( m+1 \),1_A,<Construct>Simplify an algebraic fraction by f...
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,143.0,\( m+2 \),1_B,<Construct>Simplify an algebraic fraction by f...
1,1,"Simplify the following, if possible: \( \frac{...",1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,2142.0,\( m-1 \),1_C,<Construct>Simplify an algebraic fraction by f...
2,2,Tom and Katie are discussing the \( 5 \) plant...,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,1287.0,Only\nTom,2_A,<Construct>Calculate the range from a list of ...
2,2,Tom and Katie are discussing the \( 5 \) plant...,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,1287.0,Both Tom and Katie,2_C,<Construct>Calculate the range from a list of ...
2,2,Tom and Katie are discussing the \( 5 \) plant...,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,1073.0,Neither is correct,2_D,<Construct>Calculate the range from a list of ...


# Sentence transformer models

In [30]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import gc
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

C:\Users\willv\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [31]:
#accessing HuggingFace
# Set the hub_cache dir
os.environ['HF_HOME'] = '/data/shared_models/'
#   can also be done by 
#   os.environ['TRANSFORMERS_CACHE'] = '/data/shared_models/'

# set mirror
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [32]:
BGE_path='BAAI/bge-base-en'
GTE_path='thenlper/gte-base'
MPNetV2_path='all-mpnet-base-v2'
batch_size=16

In [33]:
#
def encode_texts(test_data, misconception_mapping, model_path, batch_size=8, progress_bar=True):
    model = SentenceTransformer(model_path, local_files_only=False, trust_remote_code=True)
    model.to(device)
    # wrap the model to use all GPUs
    model = torch.nn.DataParallel(model)
    model.eval()
    
    # Encode all text from the test_data DataFrame
    all_text_vec = model.module.encode(test_data["AllText"].to_list(), batch_size=batch_size , normalize_embeddings=True, show_progress_bar=progress_bar)
    
    # Encode misconception names from the misconception_mapping DataFrame
    misconception_mapping_vec = model.module.encode(misconception_mapping["MisconceptionName"].to_list(), batch_size=batch_size, normalize_embeddings=True, show_progress_bar=progress_bar)
    
    torch.cuda.empty_cache()
    gc.collect()
    
    return model, all_text_vec, misconception_mapping_vec

In [34]:
def search_faiss(k, d, vectors_to_add, query_vectors):
    """
    Perform a FAISS search with L2 distance.
    
    Parameters:
        k (int): Number of nearest neighbors to search for.
        d (int): Dimension of the vectors.
        vectors_to_add (numpy.ndarray): The vectors to add to the FAISS index.
        query_vectors (numpy.ndarray): The vectors to search for the nearest neighbors.
        
    Returns:
        D (numpy.ndarray): The distances to the k nearest neighbors.
        I (numpy.ndarray): The indices of the k nearest neighbors.
    """
    # Create the index
    index = faiss.IndexFlatL2(d)
    
    # Add vectors to the index
    index.add(vectors_to_add)
    
    # Search for k nearest neighbors
    D, I = index.search(query_vectors, k)
    
    return D, I

# GTE-base Model

In [35]:
gte_model,gte_base_all_text_vec, gte_misconception_mapping_vec = encode_texts(expanded_test, misconcept_map, GTE_path, batch_size=batch_size)
gte_base_all_text_vec = np.pad(gte_base_all_text_vec, ((0, 0), (0, 256)), mode='constant')
gte_misconception_mapping_vec = np.pad(gte_misconception_mapping_vec, ((0, 0), (0, 256)), mode='constant')
print(gte_base_all_text_vec.shape)
print(gte_misconception_mapping_vec.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

(9, 1024)
(2587, 1024)


# MPNETV2 Model

In [36]:
mpnetv2_model,mpnetv2_all_text_vec, mpnetv2_misconception_mapping_vec = encode_texts(expanded_test, misconcept_map, MPNetV2_path, batch_size=batch_size)
mpnetv2_all_text_vec = np.pad(mpnetv2_all_text_vec, ((0, 0), (0, 256)), mode='constant')
mpnetv2_misconception_mapping_vec = np.pad(mpnetv2_misconception_mapping_vec, ((0, 0), (0, 256)), mode='constant')
print(mpnetv2_all_text_vec.shape)
print(mpnetv2_misconception_mapping_vec.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

(9, 1024)
(2587, 1024)


# BGE Model

In [37]:
bge_model,bge_all_text_vec, bge_misconception_mapping_vec = encode_texts(expanded_test, misconcept_map, BGE_path, batch_size=batch_size)
bge_all_text_vec = np.pad(bge_all_text_vec, ((0, 0), (0, 256)), mode='constant')
bge_misconception_mapping_vec = np.pad(bge_misconception_mapping_vec, ((0, 0), (0, 256)), mode='constant')
print(bge_all_text_vec.shape)
print(bge_misconception_mapping_vec.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

(9, 1024)
(2587, 1024)


# Model ensemble

In [38]:
from sklearn.ensemble import StackingClassifier

In [39]:
bge_model

DataParallel(
  (module): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
    (2): Normalize()
  )
)

In [40]:
K=25
VER=5
D = 1024
weight1, weight2, weight3 = 0.5, 0.35, 0.29

ensemble_text_vecs = (weight1 * gte_base_all_text_vec + weight2 * bge_all_text_vec + weight3 * mpnetv2_all_text_vec)

ensemble_misconception_vecs = (weight1 * gte_misconception_mapping_vec + weight2 * mpnetv2_misconception_mapping_vec + weight3 * bge_misconception_mapping_vec)

_, ensemble_indices = search_faiss(K, D, ensemble_misconception_vecs, ensemble_text_vecs)
print(ensemble_text_vecs.shape)
print(ensemble_misconception_vecs.shape)
print(ensemble_indices.shape)



(9, 1024)
(2587, 1024)
(9, 25)


In [41]:
ensemble_indices

array([[2488,  315, 2131, 2306, 1999,  871, 1872, 2586,   74, 2532, 2518,
          15, 1672, 1941, 1345, 1432,  373, 1666,  987,  466, 2221,  107,
         969, 2277,  775],
       [2488, 1999, 2131,  315, 1872,  871, 2306, 2586,   74, 2532, 2518,
        1672,   15, 1941, 1345, 1432,  987,  373, 1666, 2221,  466,  107,
         969, 2277,  775],
       [2488,  315,  871,   74, 2131, 1872, 1345,  373, 1392, 2306, 1999,
        1432, 2532, 2586,  969, 1666, 2518, 2143, 1672,   15, 1444, 1941,
        2277,  987,  107],
       [1540, 2398,  979, 1593,   59, 2307,  363,   29,  606,  633,   80,
         885,  715,  848, 1081, 2078,  891, 1980, 1469, 1825,  172,  317,
        1916,  546,    1],
       [1540, 2398,  979, 1593,   59, 2307,   29,  363,  606,  633,  885,
          80,  848,  715, 1081, 2078,  891, 1980, 1469, 1825,  172, 1916,
         317,  659,    1],
       [1540, 2398,  979, 1593,   59, 2307,  363,   29,  633,  606,   80,
         885,  715,  848, 1081,  891, 2078, 1469, 1

# Make Submit File

In [42]:
submission= pd.DataFrame(expanded_test['QuestionID_Answer'])
submission['MisconceptionID']= pd.Series(list(ensemble_indices))
submission.head()

,QuestionID_Answer,MisconceptionID
0,0_B,"[2488, 315, 2131, 2306, 1999, 871, 1872, 2586,..."
0,0_C,"[2488, 315, 2131, 2306, 1999, 871, 1872, 2586,..."
0,0_D,"[2488, 315, 2131, 2306, 1999, 871, 1872, 2586,..."
1,1_A,"[2488, 1999, 2131, 315, 1872, 871, 2306, 2586,..."
1,1_B,"[2488, 1999, 2131, 315, 1872, 871, 2306, 2586,..."


In [43]:
misconDict[315]

'Has not realised that the answer may be changed by the insertion of brackets'

# Making into a Pipeline

In [44]:
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [45]:
class preprocessor(TransformerMixin, BaseEstimator):

    def __init__(self):
        print('prep init')
        pass

    def fit(self, X, y=None):
        print('preprocess fit')
        print(X)
        return self

    def transform(self, X):
        print('preprocess transform')
        return  

In [93]:
class ensemble_nlp_model(TransformerMixin, BaseEstimator):
    BGE_path='BAAI/bge-base-en'
    GTE_path='thenlper/gte-base'
    MPNetV2_path='all-mpnet-base-v2'
    batch_size=16
    
    map_Dict={}
    
    K=10
    VER=5
    D = 1024
    weight1, weight2, weight3 = 0.5, 0.35, 0.29

    def __init__(self):
        print("nlp init")
        self.bge_model=self.init_model(BGE_path)
        self.gte_model=self.init_model(GTE_path)
        self.mpnetv2_model=self.init_model(MPNetV2_path)
        pass
    
    def init_model(self, model_path):
        model = SentenceTransformer(model_path, local_files_only=False, trust_remote_code=True)
        model.to(device)
        # wrap the model to use all GPUs
        model = torch.nn.DataParallel(model)
        model.eval()
        return model
    
    def fit(self, X, y=None):
        print('model fit')
        #print(X)
        
        self.map=X
        for i in range(self.map.shape[0]):
            self.map_Dict[self.map.MisconceptionId.values[i]] = self.map.MisconceptionName.values[i]
        
        self.map_bge_vect= self.encode_map(X,self.bge_model)
        self.map_gte_vect= self.encode_map(X,self.gte_model)
        self.map_mpnetv2_vect= self.encode_map(X,self.mpnetv2_model)
        
        
        self.map_ensemble_vect = (self.weight1 * self.map_gte_vect +
                                  self.weight2 * self.map_mpnetv2_vect +
                                  self.weight3 * self.map_bge_vect)
        return self
        
    
    def transform(self, X):
        print('nlp transform')
        
        return  X
    
    def search_faiss(self, k, d, vectors_to_add, query_vectors):
        """
        Perform a FAISS search with L2 distance.
    
        Parameters:
            k (int): Number of nearest neighbors to search for.
            d (int): Dimension of the vectors.
            vectors_to_add (numpy.ndarray): The vectors to add to the FAISS index.
            query_vectors (numpy.ndarray): The vectors to search for the nearest neighbors.
        
        Returns:
            D (numpy.ndarray): The distances to the k nearest neighbors.
            I (numpy.ndarray): The indices of the k nearest neighbors.
        """
        # Create the index
        index = faiss.IndexFlatL2(d)
    
        # Add vectors to the index
        index.add(vectors_to_add)
    
        # Search for k nearest neighbors
        D, I = index.search(query_vectors, k)
    
        return D, I
    def encode_map(self, X, model, progress_bar=True):
        # Encode misconception names from the misconception_mapping DataFrame
        misconception_mapping_vec = model.module.encode(X["MisconceptionName"].to_list(), batch_size=self.batch_size,
                                                        normalize_embeddings=True, show_progress_bar=progress_bar)
        
        torch.cuda.empty_cache()
        gc.collect()
        
        return misconception_mapping_vec
        
    def encode_texts(self, X, model, progress_bar=True):
        
        X['AllText']=X.apply(lambda x: f"<Construct>{x['ConstructName']} <Subject>{x['SubjectName']} <Question>{x['QuestionText']} <Answer>{x['AnswerText']}",axis=1)
        
        # Encode all text from X
        all_text_vec = model.module.encode(X["AllText"].to_list(), batch_size=self.batch_size ,
                                           normalize_embeddings=True, show_progress_bar=progress_bar)
    
        torch.cuda.empty_cache()
        gc.collect()
    
        return  all_text_vec
    
    def encode_text_all_models(self, X):
        self.bge_text_vec= self.encode_texts(X,self.bge_model)
        self.gte_text_vec= self.encode_texts(X,self.gte_model)
        self.mpnetv2_text_vec= self.encode_texts(X,self.mpnetv2_model)
        return self.bge_text_vec,self.gte_text_vec,self.mpnetv2_text_vec
    
    def predict(self,X):
        
        self.encode_text_all_models(X)
        
        ensemble_text_vec = (self.weight1 * self.gte_text_vec + self.weight2 * self.bge_text_vec + self.weight3 * self.mpnetv2_text_vec)
        self.D= ensemble_text_vec.shape[1]
        
        _, ensemble_indices = self.search_faiss(self.K, self.D, self.map_ensemble_vect, ensemble_text_vec)
        self.print_predict(ensemble_indices)
        pass
    
    def print_predict(self,indicies):
        count=0
        count2=0
        for i in indicies:
            count+=1
            print(f'Question {count}:')
            for j in i:
                count2+=1
                print (f"Possible Misconception {count2}: #{j}: {self.map_Dict[int(j)]}")

In [94]:
pipe = ensemble_nlp_model()

nlp init


In [95]:
pipe.fit(X=misconcept_map)

model fit


Batches:   0%|          | 0/162 [00:00<?, ?it/s]

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

Batches:   0%|          | 0/162 [00:00<?, ?it/s]

ensemble_nlp_model()

In [96]:
pipe.predict(expanded_df.iloc[[0]])

C:\Users\willv\AppData\Local\Temp\ipykernel_8716\571933905.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['AllText']=X.apply(lambda x: f"<Construct>{x['ConstructName']} <Subject>{x['SubjectName']} <Question>{x['QuestionText']} <Answer>{x['AnswerText']}",axis=1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question 1:
Possible Misconception 1: #2488: Answers order of operations questions with brackets as if the brackets are not there
Possible Misconception 2: #315: Has not realised that the answer may be changed by the insertion of brackets
Possible Misconception 3: #2131: When multiplying out 2 or more brackets, does not multiply all the combinations of terms
Possible Misconception 4: #2306: Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)
Possible Misconception 5: #1999: Does not interpret the correct order of operations from a worded problem
Possible Misconception 6: #871: Does not include brackets when attempting to multiply an expression
Possible Misconception 7: #1872: Believes they only need to multiply the first and last pairs of terms when expanding double brackets
Possible Misconception 8: #2586: Misunderstands order of operations in algebraic expressions
Possible Misconception 9: #74: Believes th

In [57]:
expanded_df.iloc[[0]]

,QuestionId,QuestionText,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,MisconceptionID,AnswerText,QuestionID_Answer,AllText
0,0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,NaN,\( 3 \times 2+(4-5) \),0_B,<Construct>Use the order of operations to carr...


In [88]:
type(misconcept_map.keys())

pandas.core.indexes.base.Index

In [92]:
count=0
count2=0
for i in ensemble_indices:
    count+=1
    print(f'Question {count}:')
    for j in i[:10]:
        count2+=1
        print (f"Possible Misconception {count2}: #{j}: {misconDict[int(j)]}")

Question 1:
Possible Misconception 1: #2488: Answers order of operations questions with brackets as if the brackets are not there
Possible Misconception 2: #315: Has not realised that the answer may be changed by the insertion of brackets
Possible Misconception 3: #2131: When multiplying out 2 or more brackets, does not multiply all the combinations of terms
Possible Misconception 4: #2306: Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)
Possible Misconception 5: #1999: Does not interpret the correct order of operations from a worded problem
Possible Misconception 6: #871: Does not include brackets when attempting to multiply an expression
Possible Misconception 7: #1872: Believes they only need to multiply the first and last pairs of terms when expanding double brackets
Possible Misconception 8: #2586: Misunderstands order of operations in algebraic expressions
Possible Misconception 9: #74: Believes th

In [100]:
expanded_df.iloc[[10]]

,QuestionId,QuestionText,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,MisconceptionID,AnswerText,QuestionID_Answer,AllText
3,3,The angles highlighted on this rectangle with ...,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,1180.0,obtuse,3_B,<Construct>Recall and use the intersecting dia...


In [104]:
print(expanded_df.iloc[[10]]['AllText'].values)

['<Construct>Recall and use the intersecting diagonals properties of a rectangle <Subject>Properties of Quadrilaterals <Question>The angles highlighted on this rectangle with different length sides can never be... ![A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.]() <Answer>obtuse']


In [98]:
pipe.predict(expanded_df.iloc[[10]])

C:\Users\willv\AppData\Local\Temp\ipykernel_8716\571933905.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['AllText']=X.apply(lambda x: f"<Construct>{x['ConstructName']} <Subject>{x['SubjectName']} <Question>{x['QuestionText']} <Answer>{x['AnswerText']}",axis=1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question 1:
Possible Misconception 1: #226: Thinks rectangles have line symmetry in the diagonals 
Possible Misconception 2: #2464: Believes the sides of a rectangle must be horizontal and vertical, not diagonal
Possible Misconception 3: #551: Believes a rectangle has four equal sides
Possible Misconception 4: #465: Does not know the diagonals of a square bisect at right angles
Possible Misconception 5: #1877: Thinks rectangles have rotational symmetry or order 4
Possible Misconception 6: #1180: Does not know the properties of a rectangle
Possible Misconception 7: #2218: Assumes that opposite angles are equal in a quadrilateral
Possible Misconception 8: #2554: Thinks the diagonal of a square is the same length as the side of the square
Possible Misconception 9: #515: Does not know a rectangle is a type of parallelogram 
Possible Misconception 10: #2174: Believes a parallelogram cannot have right angles


In [105]:
expanded_df.iloc[[100]]

,QuestionId,QuestionText,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,MisconceptionID,AnswerText,QuestionID_Answer,AllText
33,33,Water is poured into a glass at a constant rat...,1560,Given a situation sketch an appropriate real l...,58,Real Life Graphs,D,NaN,![A wide cylinder with a narrow cylinder on to...,33_B,<Construct>Given a situation sketch an appropr...


In [106]:
print(expanded_df.iloc[[100]]['AllText'].values)

['<Construct>Given a situation sketch an appropriate real life graph <Subject>Real Life Graphs <Question>Water is poured into a glass at a constant rate.\n\nThis graph shows how the depth \\( (d) \\) of water in a glass changes over time \\( (t) \\).\n\nWhich of the following glasses is this most likely to be the graph of? ![A depth-time graph which is a curve starting at (0, 0), with a steep gradient to begin with but then the gradient decreases and the graph levels off to almost horizontal.]() <Answer>![A wide cylinder with a narrow cylinder on top.]()']


In [99]:
pipe.predict(expanded_df.iloc[[100]])

C:\Users\willv\AppData\Local\Temp\ipykernel_8716\571933905.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['AllText']=X.apply(lambda x: f"<Construct>{x['ConstructName']} <Subject>{x['SubjectName']} <Question>{x['QuestionText']} <Answer>{x['AnswerText']}",axis=1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question 1:
Possible Misconception 1: #1530: Believes a steeper downwards line on a graph means a slower decrease
Possible Misconception 2: #1603: Confuses displacement-time graphs with velocity-time graphs
Possible Misconception 3: #1688: Believes a line showing a constant rate of increase corresponds to a non-prism shape
Possible Misconception 4: #284: Believes the gradient of a speed-time graph is the speed
Possible Misconception 5: #1674: Believes distance is the area under the graph on a displacement time graph
Possible Misconception 6: #919: Believes the gradient of a speed-time graph is the distance
Possible Misconception 7: #2091: Thinks depth always goes front to back in a diagram of a 3D shape
Possible Misconception 8: #559: Thinks depth is marked on the cross section of a prism
Possible Misconception 9: #2032: Believes the gradient of a distance-time graph is the acceleration
Possible Misconception 10: #1224: Does not understand the diagram given in the question
